In [ ]:
!pip install openai
import openai
openai.api_key = "your api key"
!nvidia-smi
!pip install diffusers==0.11.1
!pip install transformers scipy ftfy accelerate
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipe = pipe.to("cuda")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.3 MB/s eta 0:00:00
Thu Jun 22 18:20:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |           

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

The config attributes {'scaling_factor': 0.18215} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [ ]:
import re
def preprocess_words(string):
  response = string
  split_response = response.split("\n")

  split_response

  filtered_respone = list(filter(lambda x : x!="", split_response))

  filtered_respone

  # Remove characters and numbers using regular expressions
  filtered_list = [re.sub(r'[^a-zA-Z\s]', ' ', s) for s in filtered_respone]
  sentences = [sentence.strip() for sentence in filtered_list if sentence.strip()]

  return sentences

def split_into_words(string):
    new_string = string.split()
    return new_string

def seb(list, sebel):
  for i in list:
    if i in sebel:
      return True
  return False

def find_which_output(string, output):
  for i in string:
    for o in output:
      new_output = split_into_words(o)
      if i in new_output:
        return o

import re

def separate_topic_and_story(string):
    # Define a regular expression pattern to match the topic name
    pattern = r'^(.+?):'  # Assuming the topic ends with a colon (e.g., "Topic:")

    # Find the topic name using regex
    match = re.search(pattern, string)

    if match:
        topic = match.group(1).strip()  # Extract the topic and remove leading/trailing whitespace
        story = string[match.end():].strip()  # Extract the story part and remove leading/trailing whitespace
        return topic, story

    # If no topic name is found, return None for both
    return None, None


def split_sentence(string):
  response = string
  split_response = response.split("\n")

  split_response

  filtered_respone = list(filter(lambda x : x!="", split_response))

  return filtered_respone

In [ ]:
while True:
  inp = []
  out = []
  sebelum = ["previous","mention","mentioned","before","latest","said","wrote","write"]

  user = input("Hello, I am you AI Head Coach. You can ask me anything related to football, literally anything. Feel free to ask my any questions! ")
  completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    temperature = 0.8,
    max_tokens = 2000,
    messages = [
      {"role": "system", "content": "You are a soccer/football coach like arsene wenger, jose mourinho and pep guardiola. Answer the questions related to football or soccer only"},
      {"role": "user", "content": f"Write a brief and precise 5 points explanations about {user}. write you answers using this format, the point you are making : your explanation. make sure the point is between 3 to 5 words only."}
    ]
    )

  combined_string = " ".join(preprocess_words(completion.choices[0].message.content))
  inp.append(user)
  out.append(combined_string)
  #print(preprocess_words(completion.choices[0].message.content))
  print("QUESTION : "+user)

  for i in split_sentence(completion.choices[0].message.content):
    print("\n")
    print(i)
    topic, explains = separate_topic_and_story(i)
    #print(topic)
    generator = torch.Generator("cuda").manual_seed(1024)
    i = "Generate a vibrant colors, glow, ultra realistic, cinematic lighting, focused, 4k image of ("+str(topic)+" in soccer field). Do not put any word or sentence in the image"
    image = pipe(i, num_inference_steps=100, height=512, width=768).images[0]
    image.show()

  while True:
    satisfied_or_no = input("are you satisfied with the answer? Yes/No: ")
    if satisfied_or_no=="Yes" or satisfied_or_no=="yes":
      break
    else:
      user = input("Do you have other questions? ")
      split = split_into_words(user)
      dalam_sebelum = seb(split, sebelum)
      if dalam_sebelum==True:
          related_output = find_which_output(split, out)

          completion = openai.ChatCompletion.create(
          model = "gpt-3.5-turbo",
          temperature = 0.8,
          max_tokens = 2000,
          messages = [
          {"role": "system", "content": "You are a soccer/football coach like arsene wenger, jose mourinho and pep guardiola. Answer the questions related to football or soccer only"},
          {"role": "user", "content": f"From the sentence({related_output}), {user}? Write a brief and precise 5 points explanations for the question. write you answers using this format, the point you are making : your explanation. make sure the point is between 3 to 5 words only."}
          ]
          )

          combined_string = " ".join(preprocess_words(completion.choices[0].message.content))
          inp.append(user)

          out.append(combined_string)

          print("QUESTION : "+user)



          for i in split_sentence(completion.choices[0].message.content):
            print("\n")
            #print("ni sebab ada sebelum")
            print(i)
            topic, explains = separate_topic_and_story(i)
            #print(topic)
            generator = torch.Generator("cuda").manual_seed(1024)
            i = "Generate a vibrant colors, glow, ultra realistic, cinematic lighting, focused, 4k image of ("+str(topic)+" in soccer field). Do not put any word or sentence in the image"
            image = pipe(i, num_inference_steps=100, height=512, width=768, generator=generator).images[0]
            image.show()

      else:
          completion = openai.ChatCompletion.create(
          model = "gpt-3.5-turbo",
          temperature = 0.8,
          max_tokens = 2000,
          messages = [
          {"role": "system", "content": "You are a soccer/football coach like arsene wenger, jose mourinho and pep guardiola. Answer the questions related to football or soccer only"},
          {"role": "user", "content": f"write a brief and precise 5 points explanations about {user}. write you answers using this format, the point you are making : your explanation. make sure the point is between 3 to 5 words only."}
          ]
          )

          combined_string = " ".join(preprocess_words(completion.choices[0].message.content))
          inp.append(user)

          out.append(combined_string)

          print("QUESTION : "+user)


          for i in split_sentence(completion.choices[0].message.content):
            #print("ni sebab tak ada ")
            print("\n")
            print(i)
            topic, explains = separate_topic_and_story(i)
            #print(topic)
            generator = torch.Generator("cuda").manual_seed(1024)
            i = "Generate a vibrant colors, glow, ultra realistic, cinematic lighting, focused, 4k image of ("+str(topic)+" in soccer). Do not put any word or sentence in the image"
            image = pipe(i, num_inference_steps=100, height=512, width=768, generator=generator).images[0]
            image.show()



  print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
  print("---------------------------------------------------------------------                           ---   ----------------   ------   --------------   ------------------------------------------------------------------------------------")
  print("---------------------------------------------------------------------------------   ---------------   ----------------   --------   ----------   --------------------------------------------------------------------------------------")
  print("---------------------------------------------------------------------------------   ---------------   ----------------   ----------   ------   ----------------------------------------------------------------------------------------")
  print("---------------------------------------------------------------------------------   ---------------   ----------------   ------------   --   ------------------------------------------------------------------------------------------")
  print("---------------------------------------------------------------------------------   ---------------                      --------------    --------------------------------------------------------------------------------------------")
  print("---------------------------------------------------------------------------------   ---------------   ----------------   -----------    --   ------------------------------------------------------------------------------------------")
  print("---------------------------------------------------------------------------------   ---------------   ----------------   ----------   -----   -----------------------------------------------------------------------------------------")
  print("---------------------------------------------------------------------------------   ---------------   ----------------   --------   ---------   ---------------------------------------------------------------------------------------")
  print("---------------------------------------------------------------------------------   ---------------   ----------------   ------   -------------   -------------------------------------------------------------------------------------")
  print("---------------------------------------------------------------------------------   ---------------   ----------------   ----   -----------------   -----------------------------------------------------------------------------------")
  print("\n")
  print("\n")
  print("\n")
  print("\n")
  print("\n")
